In [1]:
%env HNSWLIB_NO_NATIVE=1 
%env ARCHFLAGS=-arch x86_64

env: HNSWLIB_NO_NATIVE=1
env: ARCHFLAGS=-arch x86_64


In [2]:
# !pip install chromadb
# !pip install langchain
# !pip install openai
# !pip install tiktoken

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [4]:
import os
import api_keys as api
os.environ["OPENAI_API_KEY"] = api.open_ai_api_key

In [5]:
from langchain.document_loaders import TextLoader, DirectoryLoader

loader = DirectoryLoader('../../docs/drugs-nhs', loader_cls=TextLoader)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [6]:
import openai
embeddings = OpenAIEmbeddings(chunk_size=1000)
docsearch = Chroma.from_documents(texts, embeddings, persist_directory="./db")

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [7]:
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [8]:
query = "explain metformin in 10 words?"
answer = qa({"query": query})
print(answer['result'], '\n')

 Metformin is a medicine used to treat type 2 diabetes. 

input_text: 
 [Document(page_content="About metformin Metformin is a medicine used to treat type 2 diabetes and gestational diabetes. It's also used to help prevent type 2 diabetes if you're at high risk of developing it. Type 2 diabetes is a condition where the body does not make enough insulin, or the insulin that it makes does not work properly. This can cause high blood sugar levels (hyperglycaemia). Metformin lowers your blood sugar levels by improving the way your body handles insulin. It's usually prescribed for diabetes when diet and exercise alone have not been enough to control your blood sugar levels. Metformin is also sometimes used to manage symptoms of polycystic ovary syndrome (PCOS), a condition that affects how the ovaries work. It is not officially approved for PCOS. Metformin treats PCOS by lowering insulin and blood sugar levels. This can also improve ovulation and encourage regular periods, even if you do no

### 1.2 Summarize all documents

In [9]:
from langchain.chains.summarize import load_summarize_chain
llm = OpenAI(temperature=0)

chain = load_summarize_chain(llm, chain_type="stuff")
print(chain.run(texts))

 Adalimumab, Allopurinol, Alogliptin, and Methotrexate are all medicines used to treat various conditions such as rheumatoid arthritis, psoriasis, Crohn's disease, and type 2 diabetes. They are all available on prescription and come in various forms such as tablets, syringes, and injection pens. They work by blocking part of the immune system to reduce inflammation or increasing the amount of insulin the body produces. Side effects may include headaches, indigestion, diarrhoea, skin rashes, and dizziness. It is important to carry a patient alert card and to tell healthcare professionals that you are taking these medicines.


### 2. Give a prompt template

In [18]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [39]:
template = """
Respond with the most easiest explanation without any difficult medical term.

QUESTION: {question}
=========
{summaries}
========="""

from langchain.prompts import PromptTemplate
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
docs = docsearch.similarity_search(query)

ans1 = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

ans1['output_text']

'\nMetformin: medicine to treat diabetes, lowers blood sugar, taken with meals, can cause side effects.'

In [40]:
template = """
Imagine that you are a doctor. You should use medical or technical terms while answering the question.

QUESTION: {question}
=========
{summaries}
========="""

from langchain.prompts import PromptTemplate
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
docs = docsearch.similarity_search(query)

ans2 = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

ans2['output_text']

'\nAnswer: Anti-diabetic drug; improves insulin action; oral/injection; lowers blood sugar; immunosuppressant; treat diabetes/PCOS; reduce inflammation; weekly dose; patient alert card.'